# Azure Data Factory - Incremental Data Load
This notebook explains how to implement an **Incremental Data Load** in Azure Data Factory (ADF) using a practical pipeline example.

#### 📌 Business Requirement
we have data comming to our order db on daily basis in sql db and we have to create one pipeline which will be able to transfer data on daily basis and incremental way with limited column in another table
EX: if 5 rows copy to storage account so next timeif 6th row present then it should ccopy only 6th row

- Data is stored in a SQL DB `orders` table.
- We need to copy data to a `orders_final` table **incrementally**.
- Only a subset of columns should be copied (e.g., exclude `last_name`).
- On re-run, only new records (based on `insert_time`) should be copied.

#### 🔄 What is Incremental Load?
- Rather than loading full data every time, we copy **only new records**.
- Helps reduce data load, improves efficiency and performance.

#### 🧠 Strategy
1. Extract the maximum `insert_time` from the **sink** table (`orders_final`).
2. In the next load, query only records from the **source** table (`orders`) with `insert_time > max_insert_time`.
3. Copy only relevant columns to the final table.

#### 🛠️ ADF Pipeline Steps
1. **Lookup Activity**: Get the latest `insert_time` from `orders_final` table.
   ```sql
   SELECT MAX(insert_time) AS date1 FROM orders_final
   ```
2. **Copy Activity**:
   - Source: SQL table `orders`
   - Query:
   ```sql
   SELECT order_id, name, insert_time
   FROM orders
   WHERE insert_time > '@{activity('Lookup1').output.firstRow.date1}'
   ```
   - Sink: SQL table `orders_final`
  
#### ✅ Sample Tables
#### Source: `orders`
| order_id | name     | last_name | insert_time          |
|----------|----------|-----------|----------------------|
| 1        | Alice    | Smith     | 2025-07-25 09:10:00  |
| 2        | Bob      | Johnson   | 2025-07-25 09:15:00  |
| 3        | Charlie  | Brown     | 2025-07-25 09:20:00  |
| 4        | Diana    | Prince    | 2025-07-26 10:00:00  |

#### Sink: `orders_final`
| order_id | name     | insert_time          |
|----------|----------|----------------------|
| 1        | Alice    | 2025-07-25 09:10:00  |
| 2        | Bob      | 2025-07-25 09:15:00  |
| 3        | Charlie  | 2025-07-25 09:20:00  |

#### 🔁 Run and Verify
- Insert new record in `orders` table.
- Run the pipeline again.
- Verify that only new records are inserted into `orders_final`.
- Example:
```sql
INSERT INTO orders VALUES (5, 'Eve', 'Evans', '2025-07-28 17:00:00');
```
Expected: Only record with order_id=5 should be inserted.

#### 📝 Notes
- Always use a reliable timestamp (`insert_time`) for incremental logic.
- Make sure timezone and format consistency is maintained.
- Consider watermark tables for production-grade pipelines.